# Why this Competition?
This competition provides an unique oppertunity for Data Science beginners to participate in a Hackathon style challenge. It also provides the unique oppertunities for beginners to get their hands dirty and indulge is practical application of ML and do one of the basic tasks machine learning algorithms are capable of doing:- Classification.

This competition has the right mix to Catergorical and Numerical features we might expect in a practical problem and this helps us know how to leverage both of thhem in conjugation for a Classification task.

# Problem Statement
The goal of this competition is to provide a fun, and approachable for anyone, tabular dataset. These competition will be great for people looking for something in between the Titanic Getting Started competition and a Featured competition.

The dataset is used for this competition is synthetic but based on a real dataset and generated using a CTGAN.

So we are sort of dealing with a variation of actual real-world data and here as Data Scientists are expected to predict the Multi-Class Classification based on these features.

## Expected Outcome:-
* Build a model to predict the category on an eCommerce product given various attributes about the listing.
* Grading Metric: Multi-Class Log Loss

## Problem Category:-
From the data and objective its is evident that this is a Multi-Class Classification Problem in the Tabular Data format.

So without further ado, let's now start with some basic imports to take us through this journey:-

In [ ]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
from scipy.optimize import fmin as scip_fmin

# Visialisation
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")

# Machine Learning

# Utils
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, cross_validate
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn import preprocessing
import category_encoders as ce

#Feature Selection
from sklearn.feature_selection import chi2, f_classif, f_regression
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile, VarianceThreshold

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier

In [ ]:
data_dir = '../input/tabular-playground-series-may-2021'

train_file_path = os.path.join(data_dir, 'train.csv')
test_file_path = os.path.join(data_dir, 'test.csv')
sample_sub_file_path = os.path.join(data_dir, 'sample_submission.csv')

print(f'Train file: {train_file_path}')
print(f'Train file: {test_file_path}')
print(f'Train file: {sample_sub_file_path}')

In [ ]:
RANDOM_SEED = 42

In [ ]:
def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [ ]:
seed_everything()

In [ ]:
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
sub_df = pd.read_csv(sample_sub_file_path)

# Descriptive Analysis

Let's do some descriptive analysis on the dataset...

In [ ]:
train_df.shape

In [ ]:
train_df.describe().T

Thus there are no missing values in any of the features... Good! Less work for us... 😄

In [ ]:
train_df.nunique()

Almost all features look like of the categorical nature with various cardinality values... Since the is no feature names we do not definitively know which ones are ordinal and which ones are nominal.  
So for sake of simplicity, let's assume all of them are nominal...

In [ ]:
ax = plt.subplots(figsize=(12, 6))
sns.set_style("whitegrid")
sns.countplot(x='target', data=train_df);
plt.ylabel("No. of Observations", size=20);
plt.xlabel("Target", size=20);

# Target Encoding

In [ ]:
mapping_dict = {'Class_1':1, 'Class_2':2, 'Class_3':3, 'Class_4':4}
train_df['target'] = train_df['target'].map(mapping_dict)

In [ ]:
train_df.dtypes

Since features are already in numric format, we do not need to convert them into numbers anymore...
# Feature Scaling
To bring all features into a similar scale let's use simple scaler to scale all the features...

In [ ]:
not_features = ['id', 'target']
features = []
for feat in train_df.columns:
    if feat not in not_features:
        features.append(feat)
print(features)

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(train_df[features])
train_df[features] = scaler.transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

# KFold Splits
Before we move on to feature engineering, it is always a good idea to perform cross validation splits. In that way, we will not risk any data leakage and would be more certain of the validation set being aptly represenative of the real world unknown data.

In [ ]:
NUM_SPLITS = 5

train_df["kfold"] = -1
train_df = train_df.sample(frac=1).reset_index(drop=True)
y = train_df.target.values
kf = StratifiedKFold(n_splits=NUM_SPLITS)
for f, (t_, v_) in enumerate(kf.split(X=train_df, y=y)):
    train_df.loc[v_, 'kfold'] = f
    
train_df.head()

# Features Selection
We need to select only the important features for better performance of the model. As unnecessary in best case scenario will not add to any productive calculation of the algorithm or in worst case scenario 'confuse' the model.

To do the same let's create a wrapper class that has all the built in statistical tests required to perform feature selection and takes some basic inputs from user and spits out the required features.

In [ ]:
# From https://github.com/abhishekkrthakur/approachingalmost
class UnivariateFeatureSelction:
    def __init__(self, n_features, problem_type, scoring, return_cols=True):
        """
        Custom univariate feature selection wrapper on
        different univariate feature selection models from
        scikit-learn.
        :param n_features: SelectPercentile if float else SelectKBest
        :param problem_type: classification or regression
        :param scoring: scoring function, string
        """
        self.n_features = n_features
        
        if problem_type == "classification":
            valid_scoring = {
                "f_classif": f_classif,
                "chi2": chi2,
                "mutual_info_classif": mutual_info_classif
            }
        else:
            valid_scoring = {
                "f_regression": f_regression,
                "mutual_info_regression": mutual_info_regression
            }
        if scoring not in valid_scoring:
            raise Exception("Invalid scoring function")
            
        if isinstance(n_features, int):
            self.selection = SelectKBest(
                valid_scoring[scoring],
                k=n_features
            )
        elif isinstance(n_features, float):
            self.selection = SelectPercentile(
                valid_scoring[scoring],
                percentile=int(n_features * 100)
            )
        else:
            raise Exception("Invalid type of feature")
    
    def fit(self, X, y):
        return self.selection.fit(X, y)
    
    def transform(self, X):
        return self.selection.transform(X)
    
    def fit_transform(self, X, y):
        return self.selection.fit_transform(X, y)
    
    def return_cols(self, X):
        if isinstance(self.n_features, int):
            mask = SelectKBest.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
                    
        elif isinstance(self.n_features, float):
            mask = SelectPercentile.get_support(self.selection)
            selected_features = []
            features = list(X.columns)
            for bool, feature in zip(mask, features):
                if bool:
                    selected_features.append(feature)
        else:
            raise Exception("Invalid type of feature")
        
        return selected_features

In [ ]:
ufs = UnivariateFeatureSelction(
    n_features=0.9,
    problem_type="classification",
    scoring="f_classif"
)

ufs.fit(train_df[features], train_df['target'].values.ravel())
selected_features = ufs.return_cols(train_df[features])

# Models Benchmarking

In [ ]:
def get_voting(vote_type='hard'):
    models = list()
    models.append(('Bernoulli', BernoulliNB(alpha=0.2)))
    models.append(('rf', RandomForestClassifier(n_estimators = 500,
                                                random_state=42)))
    models.append(('xgb', xgb.XGBClassifier(max_depth=7,
                                            n_estimators=1000,
                                            colsample_bytree=0.8,
                                            subsample=0.8,
                                            learning_rate=0.1,
                                            tree_method='gpu_hist',
                                            gpu_id=0)))
    models.append(('lgbm', LGBMClassifier(metric='multi_logloss',
                                          objective='multiclass',
                                          learning_rate=0.01,
                                          seed=42,
                                          n_estimators=1000)))
    models.append(('cbc', CatBoostClassifier(verbose=0,
                                             n_estimators=1000,
                                             eval_metric='MultiClass',
                                             task_type='GPU',
                                             devices='0',
                                             random_seed=42)))
    models.append(('mlp', MLPClassifier()))
    
    ensemble = VotingClassifier(estimators=models, voting=vote_type)
    return ensemble

In [ ]:
def get_models():
    models = dict()
    models['Bernoulli'] = BernoulliNB(alpha=0.2)
    models['rf'] = RandomForestClassifier(n_estimators = 500,
                                          random_state=42,
                                          n_jobs=-1)
    models['xgb'] = xgb.XGBClassifier(max_depth=7,
                                      n_estimators=1000,
                                      colsample_bytree=0.8,
                                      subsample=0.8,
                                      nthread=-1,
                                      learning_rate=0.1,
                                      tree_method='gpu_hist',
                                      gpu_id=0)
    models['lgbm'] = LGBMClassifier(metric='multi_logloss',
                                    objective='multiclass',
                                    seed=42,
                                    learning_rate=0.01,
                                    n_estimators=1000)
    models['cbc'] = CatBoostClassifier(verbose=0,
                                       n_estimators=1000,
                                       eval_metric='MultiClass',
                                       task_type='GPU',
                                       devices='0',
                                       random_seed=42)
    models['MLP'] = MLPClassifier()
    models['voting'] = get_voting(vote_type='soft')
    return models

def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=42)
    scores = cross_val_score(model, X, y, scoring='neg_log_loss', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [ ]:
%%time

X = train_df[selected_features]
y = train_df['target']

models = get_models()
results = []
names = []

for name, model in models.items():
    scores = -evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print(f'{name} : {round(np.mean(scores),3)} ({round(np.std(scores),3)})')

In [ ]:
ax = plt.subplots(figsize=(12, 6))
plt.boxplot(results, labels=names, showmeans=True)
plt.show()

# Submission

In [ ]:
mean_scores = []
for score in results:
    mean_scores.append(round(np.mean(score),3))
min_index = mean_scores.index(min(mean_scores))
model_name = names[min_index]

In [ ]:
print(f'Best Score: {mean_scores[min_index]}')
print(f'Best Model: {model_name}')

In [ ]:
%%time

models = get_models()
clf = models[model_name]
X = train_df[selected_features]
y = train_df['target']

clf.fit(X, y)
preds = clf.predict_proba(test_df[selected_features])
submission = pd.DataFrame()
submission['id'] = test_df['id']
submission['Class_1'] = preds[:, 0]
submission['Class_2'] = preds[:, 1]
submission['Class_3'] = preds[:, 2]
submission['Class_4'] = preds[:, 3]

In [ ]:
submission.head()

In [ ]:
submission.to_csv("ensemble_sub.csv",index=False)

**If you found this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels.** 😊